In [40]:
from post_processing import *

In [41]:
df = concatenate_playtype_seasons(play_type='DREB', seasons='All', save=True)
df = concatenate_playtype_seasons(play_type='OREB', seasons='All', save=True)

Archivo guardado en: holy_grail/labels/DREB_all.csv
Archivo guardado en: holy_grail/labels/OREB_all.csv


In [42]:
clean_missing_video_data('DREB_all.csv', True)
clean_missing_video_data('OREB_all.csv', True)

Archivo limpio guardado en: holy_grail/labels/DREB_all.csv
Archivo limpio guardado en: holy_grail/labels/OREB_all.csv


Download frames from vídeo

In [ ]:
import imageio.v3 as iio
import os
import cv2
import numpy as np
from tqdm import tqdm

def extract_frames(video_path, output_folder, fps=30, resolution=None):
    os.makedirs(output_folder, exist_ok=True)
    
    frames = iio.imread(video_path, plugin="pyav", index=None)  # Cargar todos los frames
    total_frames = len(frames[::int(30/fps)])
    
    for i, frame in tqdm(enumerate(frames[::int(30/fps)]), total=total_frames, desc="Extrayendo frames"):
        if resolution:
            frame = cv2.resize(frame, resolution, interpolation=cv2.INTER_AREA)
        
        image_path = os.path.join(output_folder, f"frame_{i:04d}.png")
        iio.imwrite(image_path, frame)
    
    print(f"Frames guardados en {output_folder}")

extract_frames("ignore/example_video.mp4", "frames", fps=30, resolution=(640, 360))


Extrayendo frames: 100%|██████████| 621/621 [00:09<00:00, 67.95it/s]

Frames guardados en frames


Count unique players in the dataset

In [8]:
import pandas as pd

def count_unique_players(file_path1, file_path2):
    df1 = pd.read_csv(file_path1)
    df2 = pd.read_csv(file_path2)
    combined_df = pd.concat([df1, df2])
    unique_players = combined_df['PLAYER NAME'].nunique()
    return unique_players

# Ejemplo de uso
file_path1 = 'holy_grail/labels/OREB_all.csv'
file_path2 = 'holy_grail/labels/DREB_all.csv'
print(f"El número de jugadores distintos es: {count_unique_players(file_path1, file_path2)}")

El número de jugadores distintos es: 817


In [15]:


directorio_actual = os.getcwd()
print(directorio_actual)

/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA


In [17]:
import pandas as pd

def add_downloaded_column():
    # Leer los archivos CSV
    oreb_df = pd.read_csv("holy_grail/labels/OREB_all.csv")
    dreb_df = pd.read_csv("holy_grail/labels/DREB_all.csv")
    
    # Añadir la columna 'DOWNLOADED' con valores False
    oreb_df["DOWNLOADED"] = False
    dreb_df["DOWNLOADED"] = False
    
    # Sobrescribir los archivos con la nueva columna
    oreb_df.to_csv("holy_grail/labels/OREB_all.csv", index=False)
    dreb_df.to_csv("holy_grail/labels/DREB_all.csv", index=False)
    
    print("Columna 'DOWNLOADED' añadida y archivos sobrescritos exitosamente.")

# Llamar a la función
add_downloaded_column()


Columna 'DOWNLOADED' añadida y archivos sobrescritos exitosamente.


In [2]:
oreb = pd.read_csv('HoopCut_net/OREB_all.csv')
dreb = pd.read_csv('HoopCut_net/DREB_all.csv')

In [25]:
import pandas as pd

def process_csv_files(file1, file2, output_file):
    # Leer los archivos CSV
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    
    # Lista para almacenar las filas seleccionadas
    selected_rows = []
    
    # Función para seleccionar filas aleatorias y actualizar 'DOWNLOADED'
    def select_random_rows(df):
        # Obtener valores únicos de la columna 'SEASON'
        unique_seasons = df['SEASON'].unique()
        
        for season in unique_seasons:
            # Filtrar filas por 'SEASON'
            season_rows = df[df['SEASON'] == season]
            # Seleccionar 25 filas aleatorias
            random_rows = season_rows.sample(n=min(25, len(season_rows)), random_state=1)
            # Establecer 'DOWNLOADED' a True
            df.loc[random_rows.index, 'DOWNLOADED'] = True  # Actualizar directamente el DataFrame original
            # Añadir las filas seleccionadas a la lista
            selected_rows.append(random_rows)
    
    # Seleccionar filas de ambos DataFrames y actualizar 'DOWNLOADED'
    select_random_rows(df1)
    select_random_rows(df2)
    
    # Concatenar todas las filas seleccionadas en un nuevo DataFrame
    selected_df = pd.concat(selected_rows, ignore_index=True)
    
    # Guardar los DataFrames actualizados en sus archivos originales
    df1.to_csv(file1, index=False)
    df2.to_csv(file2, index=False)
    
    # Guardar el nuevo DataFrame con las filas seleccionadas en un archivo CSV
    selected_df.to_csv(output_file, index=False)

# Uso de la función
file1 = 'HoopCut_net/DREB_all.csv'  # Cambia esto por el nombre de tu archivo
file2 = 'HoopCut_net/OREB_all.csv'  # Cambia esto por el nombre de tu archivo
output_file = 'HoopCut_net/labels_HoopCut.csv'  # Archivo de salida para las filas seleccionadas

process_csv_files(file1, file2, output_file)


In [45]:
import pandas as pd

def count_true_values(df, column_name):
    """
    Cuenta cuántos valores True hay en una columna dada de un DataFrame.

    :param df: DataFrame de pandas.
    :param column_name: Nombre de la columna donde contar los valores True.
    :return: Número de valores True en la columna especificada.
    """
    if column_name in df.columns:
        return df[column_name].sum()  # Suma los valores True (1) en la columna
    else:
        raise ValueError(f"La columna '{column_name}' no existe en el DataFrame.")

# Ejemplo de uso
df = pd.read_csv('HoopCut_net/DREB_all.csv')
count_true = count_true_values(df, 'DOWNLOADED')
print(count_true)


250


In [35]:
import pandas as pd

def count_string_occurrences(df, column_name, target_string):
    """
    Cuenta cuántas veces aparece una cadena específica en una columna dada de un DataFrame.

    :param df: DataFrame de pandas.
    :param column_name: Nombre de la columna donde contar las ocurrencias.
    :param target_string: Cadena a contar en la columna.
    :return: Número de ocurrencias de la cadena en la columna especificada.
    """
    if column_name in df.columns:
        return (df[column_name] == target_string).sum()  # Suma las ocurrencias de la cadena
    else:
        raise ValueError(f"La columna '{column_name}' no existe en el DataFrame.")

# Ejemplo de uso
df = pd.read_csv(output_file)
count_occurrences = count_string_occurrences(df, 'REBOUND TYPE', 'DREB')
print(count_occurrences)


250


In [36]:
import pandas as pd

def add_play_id_column(file_path, output_file):
    """
    Lee un archivo CSV, añade una columna 'PLAY ID' y desplaza las demás columnas hacia la derecha.

    :param file_path: Ruta del archivo CSV original.
    :param output_file: Ruta del archivo CSV de salida con la nueva columna.
    """
    # Leer el archivo CSV
    df = pd.read_csv(file_path)

    # Crear la nueva columna 'PLAY ID' con el formato deseado
    df.insert(0, 'PLAY ID', [f'hoop_cut_{i+1}' for i in range(len(df))])

    # Guardar el DataFrame actualizado en un nuevo archivo CSV
    df.to_csv(output_file, index=False)

# Ejemplo de uso
input_file = output_file  # Cambia esto por el nombre de tu archivo
output_file = output_file  # Archivo de salida
add_play_id_column(input_file, output_file)


In [1]:
from download import download_videos_from_dataframe
import pandas as pd

df = pd.read_csv('HoopCut_net/labels_HoopCut.csv')

download_videos_from_dataframe(df, 'HoopCut_net/dataset', max_workers=32)

/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Descargando vídeos:  65%|██████▍   | 323/500 [00:38<00:10, 17.33it/s]

Error al descargar HoopCut_net/dataset/hoop_cut_373.mp4: Invalid URL 'False': No scheme supplied. Perhaps you meant https://False?


Descargando vídeos: 100%|██████████| 500/500 [00:58<00:00,  8.60it/s]


In [51]:
oreb = 'output.csv'
dreb = 'holy_grail/labels/DREB_all.csv'

import pandas as pd

def count_unique_values(csv_file):
    # Leer el archivo CSV
    df = pd.read_csv(csv_file)

    # Verificar si las columnas existen en el DataFrame
    columns_to_check = ["PLAY DESCRIPTION", "REBOUND TYPE"]
    missing_columns = [col for col in columns_to_check if col not in df.columns]
    
    if missing_columns:
        raise ValueError(f"Las siguientes columnas no están en el archivo: {', '.join(missing_columns)}")

    # Obtener valores únicos
    unique_play_type = df["PLAY DESCRIPTION"].dropna().unique()
    unique_rebound_type = df["REBOUND TYPE"].dropna().unique()

    # Contar valores únicos
    play_type_count = len(unique_play_type)
    rebound_type_count = len(unique_rebound_type)

    # Crear un DataFrame con los valores distintos
    unique_values_df = pd.DataFrame({
        "PLAY TYPE": list(unique_play_type) + [""] * (max(play_type_count, rebound_type_count) - play_type_count),
        "REBOUND TYPE": list(unique_rebound_type) + [""] * (max(play_type_count, rebound_type_count) - rebound_type_count)
    })

    return play_type_count, rebound_type_count, unique_values_df

# Ejemplo de uso
play_count, rebound_count, unique_df = count_unique_values(oreb)

print(f"Valores únicos en 'PLAY TYPE': {play_count}")
print(f"Valores únicos en 'REBOUND TYPE': {rebound_count}")
print(unique_df)


Valores únicos en 'PLAY TYPE': 16345
Valores únicos en 'REBOUND TYPE': 1
                             PLAY TYPE REBOUND TYPE
0      Westbrook REBOUND (Off:1 Def:0)         OREB
1      Westbrook REBOUND (Off:2 Def:0)             
2      Westbrook REBOUND (Off:1 Def:1)             
3      Westbrook REBOUND (Off:2 Def:1)             
4      Westbrook REBOUND (Off:3 Def:2)             
...                                ...          ...
16340     Looney REBOUND (Off:3 Def:1)             
16341     Looney REBOUND (Off:4 Def:1)             
16342     Looney REBOUND (Off:2 Def:5)             
16343     Looney REBOUND (Off:3 Def:5)             
16344     Looney REBOUND (Off:4 Def:7)             

[16345 rows x 2 columns]


In [49]:
import pandas as pd

def swap_columns_in_range(csv_file, start_row, end_row, output_file="output.csv"):
    # Leer el archivo CSV
    df = pd.read_csv(csv_file)

    # Verificar si las columnas existen en el DataFrame
    columns_to_swap = ["PLAY DESCRIPTION", "REBOUND TYPE"]
    missing_columns = [col for col in columns_to_swap if col not in df.columns]
    
    if missing_columns:
        raise ValueError(f"Las siguientes columnas no están en el archivo: {', '.join(missing_columns)}")

    # Asegurar que start_row y end_row están dentro del rango del DataFrame
    if start_row < 0 or end_row >= len(df):
        raise IndexError("El rango de filas está fuera de los límites del DataFrame.")

    # Intercambiar los valores de las dos columnas en el rango de filas
    temp = df.loc[start_row:end_row, "PLAY DESCRIPTION"].copy()
    df.loc[start_row:end_row, "PLAY DESCRIPTION"] = df.loc[start_row:end_row, "REBOUND TYPE"]
    df.loc[start_row:end_row, "REBOUND TYPE"] = temp

    # Guardar el archivo modificado
    df.to_csv(output_file, index=False)

    return df

# Ejemplo de uso
path_holy = "HoopCut_net/"
csv_file = path_holy + "OREB_all.csv"
start_row = 60825   # Fila donde comienza el intercambio
end_row = 65426     # Fila donde termina el intercambio

df_modificado = swap_columns_in_range(csv_file, start_row, end_row)
print(df_modificado.loc[start_row:end_row])  # Mostrar las filas afectadas


        SEASON  PLAYER ID   PLAYER NAME REBOUND TYPE  \
60825  2018-19         42    Kyle Kuzma         OREB   
60826  2018-19         42    Kyle Kuzma         OREB   
60827  2018-19         42    Kyle Kuzma         OREB   
60828  2018-19         42    Kyle Kuzma         OREB   
60829  2018-19         42    Kyle Kuzma         OREB   
...        ...        ...           ...          ...   
65422  2018-19         95  Steven Adams         OREB   
65423  2018-19         95  Steven Adams         OREB   
65424  2018-19         95  Steven Adams         OREB   
65425  2018-19         95  Steven Adams         OREB   
65426  2018-19         95  Steven Adams         OREB   

                  PLAY DESCRIPTION   BOXSCORE  VTM  HTM  \
60825  Kuzma REBOUND (Off:1 Def:1)  HOU @ LAL  HOU  LAL   
60826  Kuzma REBOUND (Off:1 Def:0)  LAL @ PHX  LAL  PHX   
60827  Kuzma REBOUND (Off:1 Def:1)  LAL @ SAS  LAL  SAS   
60828  Kuzma REBOUND (Off:1 Def:3)  LAL @ MIN  LAL  MIN   
60829  Kuzma REBOUND (Off:2 Def:

In [52]:
import cv2
import os

def check_fps_in_folder(folder_path, target_fps=60):
    """
    Revisa si todos los videos en una carpeta tienen la tasa de FPS esperada.

    :param folder_path: Ruta de la carpeta que contiene los videos.
    :param target_fps: FPS esperado (por defecto 60).
    :return: Lista con videos que NO cumplen con los FPS y sus valores reales.
    """
    non_matching_videos = []  # Almacena videos que no cumplen la condición
    
    # Obtener todos los archivos de la carpeta
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        
        # Verificar si es un archivo de video
        if os.path.isfile(file_path) and file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv', '.flv')):
            cap = cv2.VideoCapture(file_path)
            if not cap.isOpened():
                print(f"⚠️ No se pudo abrir el archivo: {file}")
                continue
            
            fps = cap.get(cv2.CAP_PROP_FPS)  # Obtener FPS del video
            cap.release()
            
            if fps != target_fps:
                non_matching_videos.append((file, fps))  # Guardar videos que no cumplen

    # Mostrar resultados
    if non_matching_videos:
        print("❌ Algunos videos no cumplen con los FPS esperados:")
        for video, fps in non_matching_videos:
            print(f"  - {video}: {fps} FPS")
    else:
        print("✅ Todos los videos tienen la tasa de FPS correcta.")

    return non_matching_videos  # Retorna la lista de videos fuera del estándar

# 📌 Ejemplo de uso
folder_path = "HoopCut_net/dataset"
check_fps_in_folder(folder_path)


✅ Todos los videos tienen la tasa de FPS correcta.


[]